In [1]:
from read_user_timeline import read_user_timeline
import pickle
import os
import pandas as pd

In [2]:
office = ['Microsoft365','MSFTDynamics365','linkedin']
computing = ['Xbox','surface','windows']
cloud = ['Azure','SQLServer','windowsserver','github']

In [3]:
if(os.path.isfile('@Microsoft365tweets.sav')==False):
    print('hello')

hello


In [9]:
tweets_office=[]
for thandle in office:
    if(os.path.isfile(thandle+'tweets.sav')==False):
        print('Extracting timeline for ' +thandle)
        read_user_timeline(thandle, 'tweepy')
        tweets_office.append(pickle.load(open(thandle+'tweets.sav','rb')))
    else:
        print('Loading previously extracted timeline for ' + thandle)
        tweets_office.append(pickle.load(open(thandle+'tweets.sav','rb')))

Loading previously extracted timeline for Microsoft365
Loading previously extracted timeline for MSFTDynamics365
Loading previously extracted timeline for linkedin


In [10]:
date_range = []
date_list = []
text_list = []
for i in range(len(tweets_office)):
    dates = [t['created_at'] for t in tweets_office[i]]
    dates = pd.to_datetime(dates)
    date_list.append(dates)
    text = [t['text'] for t in tweets_office[i]]
    text_list.append(text)
    tmp=[]
    tmp.append(dates.min())
    tmp.append(dates.max())
    date_range.append(tmp)

In [11]:
date_range

[[Timestamp('2020-02-20 12:28:22+0000', tz='UTC'),
  Timestamp('2020-09-17 18:00:05+0000', tz='UTC')],
 [Timestamp('2019-08-21 20:46:00+0000', tz='UTC'),
  Timestamp('2020-09-17 18:30:02+0000', tz='UTC')],
 [Timestamp('2018-02-23 23:48:00+0000', tz='UTC'),
  Timestamp('2020-09-17 16:18:35+0000', tz='UTC')]]

In [12]:
date_list[0].shape

(2000,)

In [13]:
df = pd.DataFrame(date_list[0],columns=['Date'])
df.shape

(2000, 1)

In [15]:
df['Text'] = text_list[0]

In [16]:
df.head()

,Date,Text
0,2020-09-17 18:00:05+00:00,"On September 22, join our engineering, product..."
1,2020-09-17 16:37:50+00:00,"@hiteshsamvedi Hi there, we're sorry to hear t..."
2,2020-09-17 16:29:41+00:00,"@Jonatha48569290 Hi Jonathan, thanks for reach..."
3,2020-09-17 16:24:02+00:00,@CTSLAFRICA Great choice! 👍
4,2020-09-17 15:00:13+00:00,How do you deliver meaningful customer engagem...
